In [1]:
import pandas as pd
import re
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from preprocssing import preprocessDF , get_vocab
from TD_IDF import tfidf_transformation
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import metrics

In [2]:
#preprocessing on training data set
train_tokens = []
train_labels = []
train_tokens, train_labels =preprocessDF('./dataset/train.csv', 'train')


In [3]:
# store tweets after preprocessing in data frame to extract tweets and labels
df_train_processed= pd.read_csv("train_processed.csv")
train_tweets = df_train_processed['text'].values
train_labels= df_train_processed['stance'].values
#extract vocabulary from training set
vocab =set()
vocab=list(get_vocab(train_tokens))
N = len(train_tweets)# no of tweets= 6988
V = len(vocab) # no of unique words= 26843



# Train and Validation 


In [6]:
# VALIDATION DATA PREPROCESSING
val_tokens = []
val_labels = []
val_tokens, val_labels =preprocessDF('./dataset/dev.csv','val')
df_val_processed= pd.read_csv("val_processed.csv")
val_tweets = df_val_processed['text'].values
val_labels= df_val_processed['stance'].values
print(len(val_tweets))


1000


In [7]:
cv=CountVectorizer() 
# this steps generates word counts for the words in your docs 

''' 
The fit method is calculating the mean and variance of each of the features present in our data. 
The transform method is transforming all the features using the respective mean and variance.

Now, we want scaling to be applied to our test data too and at the same time do not want to be 
biased with our model. We want our test data to be a completely new and a surprise set for our
 model. The transform method helps us in this case.
'''
#create tf matrix of size (N*V) 
word_count_vector=cv.fit_transform(train_tweets)
print(word_count_vector.shape)
#idf(t) = log [ n / df(t) ] + 1  
# smoothed ->, terms that occur in all documents in a training set, will not be entirely ignored.
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
#use count vector to compute IDf
tfidf_transformer.fit(word_count_vector) # TODO Fit wla fit transform
tfidf_transformer.transform(val_tweets)
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])
# compute TF for each tweet that generate Matrix of size (N*V) 
# each row represent a tweet, each col a word the cell has count of this word in that tweet
count_vector=cv.transform(train_tweets) 
#print(count_vector.shape)
# tf-idf scores using TF matrix and IDFs valued that we fit the tfidf_transformer object using it

X = cv.transform(val_tweets)
tf_idf_val =tfidf_transformer.transform(X, copy=False)
tf_idf=tfidf_transformer.transform(count_vector)

(6988, 27078)


c:\Users\dell\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
feature_names = cv.get_feature_names() 
# print(feature_names)
#get tfidf vector for first document 
first_document_vector=tf_idf_train[1] 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

c:\Users\dell\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,tfidf
انوا,0.344647
نفاق,0.186433
معوا,0.186433
بمؤتمروا,0.186433
عفوا,0.186433
...,...
بلغثيان,0.000000
بلغت,0.000000
بلغ,0.000000
بلعيبه,0.000000


In [ ]:
# word_count_vector=cv.transform(val_tweets)
# print(word_count_vector.shape)
# #idf(t) = log [ n / df(t) ] + 1  
# smoothed ->, terms that occur in all documents in a training set, will not be entirely ignored.
# tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
# tfidf_transformer.transform(word_count_vector) # TODO Fit wla fit transform
# print idf values 
# df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
# sort ascending 
# df_idf.sort_values(by=['idf_weights'])
# compute TF for each tweet that generate Matrix of size (N*V) 
# each row represent a tweet, each col a word the cell has count of this word in that tweet
#count_vector=cv.transform(val_tweets) 
#print(count_vector.shape)
# tf-idf scores using TF matrix and IDFs valued that we fit the tfidf_transformer object using it
#tf_idf_val=tfidf_transformer.transform(count_vector)


NotFittedError: idf vector is not fitted

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit(train_tweets)#mode
X.transform(train_tweets)
X.transform(val_tweets)
#-----------------------------------
word_count_vector=cv.fit_transform()
tfidf_transformer_model=TfidfTransformer(smooth_idf=True,use_idf=True) 
#use count vector to compute IDf
tfidf_transformer_model.fit(tweets) # TODO Fit wla fit transform
tfidf_transformer_model.transform(val_tweets)

In [ ]:
clf=LinearSVC(C=50.0, random_state=42)
clf.fit(tf_idf_train,train_labels )
Y_Predicted=clf.predict(tf_idf_val)
print("Accuracy:",metrics.accuracy_score(val_labels, Y_Predicted)*100,"%")

c:\Users\dell\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


ValueError: X has 7273 features, but LinearSVC is expecting 27078 features as input.